<a href="https://colab.research.google.com/github/S-ArunaDevi06/AI_generated_review_detection/blob/main/AI_Human_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip uninstall transformers
!pip install torch --upgrade
!pip install transformers --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 736.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers datasets

Tamil

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from datasets import Dataset
from sklearn.metrics import f1_score

# Load CSV from Google Drive
file_path = "/content/drive/MyDrive/Human_AI_detection/tam_training_data_hum_ai.csv"  # Adjust your path
data = pd.read_csv(file_path)

# Preprocessing: Extract relevant columns
data = data[['DATA', 'LABEL']]
data['LABEL'] = data['LABEL'].astype('category').cat.codes  # Convert labels to numeric

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['DATA'], data['LABEL'], test_size=0.25, random_state=42
)

# Load tokenizer and model
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=data['LABEL'].nunique())

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/606 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/202 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to eval_strategy
    logging_dir='./logs',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # Prevents reporting to W&B
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Use NumPy for logits processing

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute macro F1 score
    macro_f1 = f1_score(labels, predictions, average='macro')

    return {"accuracy": accuracy, "macro_f1": macro_f1}


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


<ipython-input-5-8e6bc979d940>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.543700,0.444697,0.955446,0.955128
2,0.390500,0.383266,0.960396,0.960146
3,0.336000,0.361136,0.965347,0.965154


TrainOutput(global_step=114, training_loss=0.4474204140796996, metrics={'train_runtime': 903.4711, 'train_samples_per_second': 2.012, 'train_steps_per_second': 0.126, 'total_flos': 3406132964520.0, 'train_loss': 0.4474204140796996, 'epoch': 3.0})

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer

# Load your test dataset
test_dataset = pd.read_excel("/content/drive/MyDrive/Human_AI_detection/tam_test_data_hum_ai.xlsx")

# Convert the DataFrame to a HuggingFace Dataset
test_hf_dataset = Dataset.from_pandas(test_dataset)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/content-type-classifier-deberta")

# Tokenize the test dataset
test_hf_dataset = test_hf_dataset.map(lambda x: tokenizer(x['DATA'], truncation=True, padding=True), batched=True)

# Get model predictions
predictions = trainer.predict(test_hf_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)


# Convert numeric predictions to original labels
pred_labels_mapped = []
for i in range(len(pred_labels)):
  if pred_labels[i]==1:
    pred_labels_mapped.append("HUMAN")
  else:
    pred_labels_mapped.append("AI")


print(pred_labels_mapped)
# Create a DataFrame with ID and PREDICTION
results_df = pd.DataFrame({
    "ID": test_dataset['ID'],
    "PREDICTION": pred_labels_mapped
})


print(results_df)
# Save as a TSV file
results_df.to_csv("tamil_indicBERT_test_results.tsv", sep='\t', index=False)

print("Results saved to 'tamil_indicBERT_test_results.tsv'")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

['AI', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'HUMAN', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'HUMAN', 'AI', 'AI', 'HUMAN', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'AI', 'HUMAN', 'AI', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'AI', 'HUMAN', 'HUMAN', 'HUMAN', 'AI']
                 ID PREDICTION
0   TAM_HUAI_TE_001         AI
1   TAM_HUAI_TE_002      HUMAN
2   TAM_HUAI_TE_003      HUMAN
3   TAM_HUAI_TE_004      HUMAN
4   TAM_HUAI_TE_005      HUMAN
..              ...        ...
95  TAM_HUAI_TE_096         AI
96  TAM

In [ ]:
# Evaluate on test set
results = trainer.evaluate(eval_dataset=test_dataset)

# Print macro F1 score
print(f"Macro F1 Score: {results['eval_macro_f1']}")
print(f"Accuracy: {results} ")


Macro F1 Score: 0.9650993459212638
Accuracy: {'eval_loss': 0.3436174988746643, 'eval_accuracy': 0.9653465346534653, 'eval_macro_f1': 0.9650993459212638, 'eval_runtime': 34.1562, 'eval_samples_per_second': 5.914, 'eval_steps_per_second': 0.381, 'epoch': 3.0} 


Malayalam


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from datasets import Dataset
from sklearn.metrics import f1_score

# Load CSV from Google Drive
file_path = "/content/drive/MyDrive/Human_AI_detection/mal_training_data_hum_ai.csv"  # Adjust your path
data = pd.read_csv(file_path)

# Preprocessing: Extract relevant columns
data = data[['DATA', 'LABEL']]
data['LABEL'] = data['LABEL'].astype('category').cat.codes  # Convert labels to numeric

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['DATA'], data['LABEL'], test_size=0.25, random_state=42
)

# Load tokenizer and model
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=data['LABEL'].nunique())

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Updated to eval_strategy
    logging_dir='./logs',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to="none"  # Prevents reporting to W&B
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Use NumPy for logits processing

    # Compute accuracy
    accuracy = accuracy_score(labels, predictions)

    # Compute macro F1 score
    macro_f1 = f1_score(labels, predictions, average='macro')

    return {"accuracy": accuracy, "macro_f1": macro_f1}


# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


<ipython-input-19-8e6bc979d940>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.648000,0.569616,0.730000,0.712000
2,0.387300,0.345607,0.950000,0.949955


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.648000,0.569616,0.730000,0.712000
2,0.387300,0.345607,0.950000,0.949955
3,0.293100,0.308776,0.965000,0.964929


TrainOutput(global_step=114, training_loss=0.46726180064050776, metrics={'train_runtime': 1878.3071, 'train_samples_per_second': 0.958, 'train_steps_per_second': 0.061, 'total_flos': 7852766875200.0, 'train_loss': 0.46726180064050776, 'epoch': 3.0})

In [ ]:
# Evaluate on test set
results = trainer.evaluate(eval_dataset=test_dataset)

# Print macro F1 score
print(f"Macro F1 Score: {results['eval_macro_f1']}")
print(f"Accuracy: {results} ")


Macro F1 Score: 0.9649289811869035
Accuracy: {'eval_loss': 0.3087761104106903, 'eval_accuracy': 0.965, 'eval_macro_f1': 0.9649289811869035, 'eval_runtime': 76.6079, 'eval_samples_per_second': 2.611, 'eval_steps_per_second': 0.17, 'epoch': 3.0} 


In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer

# Load your test dataset
test_dataset = pd.read_excel("/content/drive/MyDrive/Human_AI_detection/mal_test_data_hum_ai.xlsx")

# Convert the DataFrame to a HuggingFace Dataset
test_hf_dataset = Dataset.from_pandas(test_dataset)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("nvidia/content-type-classifier-deberta")

# Tokenize the test dataset
test_hf_dataset = test_hf_dataset.map(lambda x: tokenizer(x['DATA'], truncation=True, padding=True), batched=True)

# Get model predictions
predictions = trainer.predict(test_hf_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
print(pred_labels)

# Convert numeric predictions to original labels
pred_labels_mapped = []
for i in pred_labels:
  if i==1:
    pred_labels_mapped.append("HUMAN")
  else:
    pred_labels_mapped.append("AI")


print(pred_labels_mapped)
# Create a DataFrame with ID and PREDICTION
results_df = pd.DataFrame({
    "ID": test_dataset['ID'],
    "PREDICTION": pred_labels_mapped
})


# Save as a TSV file
results_df.to_csv("malayalam_indicBERT_test_results.tsv", sep='\t', index=False)

print("Results saved to 'malayalam_indicBERT_test_results.tsv'")


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
['HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN', 'HUMAN'

In [ ]:
print(data)

                                                  DATA  LABEL
0    ഞാൻ കുറച്ച് കാലമായി മുച്ചട്ച്ചിൻ്റെ ഫേസ് വാഷ് ...      1
1             ഈ ഫേസ് വാഷ് തണുപ്പ് വെതറിലും ഉപയോഗിക്കാം      1
2    അണ്ണാ എനിക്ക് 14 വയസ് ആയ തേയോളു എനിക്ക് സ്കിൻക...      1
3    ബ്രോ ഇതെല്ലം യൂസ്  ആക്കീട്ട് നൈറ്റ് പിന്നെ വേറ...      1
4      ഇത് ഫേസ് വാഷ് ഡെയിലി ചെയ്താ സ്കിൻകെയറിന് നല്ലതാ      1
..                                                 ...    ...
795  ബിരിയാണി, പപ്പടം, അച്ചാർ - മറ്റെവിടെയും കിട്ടാ...      0
796  എങ്കിലും, തട്ടുകടയിലെ ഭക്ഷണത്തിന്റെ സുഖം മറ്റൊ...      0
797  പോറോട്ട, ബീഫ് കറി, സാലഡ് - ഈ കോമ്പിനേഷനിൽ നിന്...      0
798  നല്ല ഉഴുന്നുവട്ടിയും, കിടിലൻ ചമ്മന്തിയും ചേർന്...      0
799  അങ്ങോട്ട് പോയാൽ, ചിക്കൻ കറി, ചപ്പാത്തി ഒന്നും ...      0

[800 rows x 2 columns]


# **SUPPORT VECTOR MACHINE**


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer


# Load CSV from Google Drive
file_path = "/content/drive/MyDrive/Human_AI_detection/tam_training_data_hum_ai.csv"  # Adjust your path
data = pd.read_csv(file_path)

# Preprocessing: Extract relevant columns
data = data[['DATA', 'LABEL']]
data['LABEL'] = data['LABEL'].astype('category').cat.codes  # Convert labels to numeric


# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_df=0.7)

# Transform the text data to feature vectors
X = vectorizer.fit_transform(data['DATA'])

# Labels
y = data['LABEL']


# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, test_size=0.25, random_state=42
)




In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

clf = SVC(kernel='linear')
clf.fit(train_texts,train_labels)

y_pred = clf.predict(test_texts)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.8465346534653465
F1 Score: 0.8465007966662581


tamil svm

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Load your datasets
train_dataset = pd.read_csv("/content/drive/MyDrive/Human_AI_detection/tam_training_data_hum_ai.csv")
test_dataset = pd.read_excel("/content/drive/MyDrive/Human_AI_detection/tam_test_data_hum_ai.xlsx")

# Vectorize the train and test data
vectorizer = TfidfVectorizer(max_df=0.7)

# Fit the vectorizer on the train data
X_train = vectorizer.fit_transform(train_dataset['DATA'])
y_train = train_dataset['LABEL']

# Transform the test data
X_test = vectorizer.transform(test_dataset['DATA'])

# Train the SVM model
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Convert numeric predictions to original labels
#pred_labels_mapped = ["HUMAN" if i == 1 else "AI" for i in y_pred]

# Create a DataFrame with ID and PREDICTION
results_df = pd.DataFrame({
    "ID": test_dataset['ID'],
    "PREDICTION": y_pred
})

# Save the results as a TSV file
results_df.to_csv("tamil_SVM_test_results.tsv", sep='\t', index=False)

print("Results saved to 'tamil_SVM_test_results.tsv'")


Results saved to 'tamil_SVM_test_results.tsv'


malayalam svm

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC


# Load CSV from Google Drive
file_path = "/content/drive/MyDrive/Human_AI_detection/mal_training_data_hum_ai.csv"  # Adjust your path
data = pd.read_csv(file_path)

# Preprocessing: Extract relevant columns
data = data[['DATA', 'LABEL']]
data['LABEL'] = data['LABEL'].astype('category').cat.codes  # Convert labels to numeric


# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_df=0.7)

# Transform the text data to feature vectors
X = vectorizer.fit_transform(data['DATA'])

# Labels
y = data['LABEL']


# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    X, y, test_size=0.25, random_state=42
)





clf = SVC(kernel='linear')
clf.fit(train_texts,train_labels)

y_pred = clf.predict(test_texts)

accuracy = accuracy_score(test_labels, y_pred)
f1 = f1_score(test_labels, y_pred, average='macro')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.775
F1 Score: 0.7748592870544091


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

# Load your datasets
train_dataset = pd.read_csv("/content/drive/MyDrive/Human_AI_detection/mal_training_data_hum_ai.csv")
test_dataset = pd.read_excel("/content/drive/MyDrive/Human_AI_detection/mal_test_data_hum_ai.xlsx")

# Vectorize the train and test data
vectorizer = TfidfVectorizer(max_df=0.7)

# Fit the vectorizer on the train data
X_train = vectorizer.fit_transform(train_dataset['DATA'])
y_train = train_dataset['LABEL']

# Transform the test data
X_test = vectorizer.transform(test_dataset['DATA'])

# Train the SVM model
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Convert numeric predictions to original labels
#pred_labels_mapped = ["HUMAN" if i == 1 else "AI" for i in y_pred]

# Create a DataFrame with ID and PREDICTION
results_df = pd.DataFrame({
    "ID": test_dataset['ID'],
    "PREDICTION": y_pred
})

# Save the results as a TSV file
results_df.to_csv("malayalam_SVM_test_results.tsv", sep='\t', index=False)

print("Results saved to 'malayalam_SVM_test_results.tsv'")


Results saved to 'malayalam_SVM_test_results.tsv'


# METRICS FOR TEST DATA AFTER THE LABELS HAVE BEEN PROVIDED

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score


tam_actual_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/tam_test_with_labels.csv')
mal_actual_df=  pd.read_csv('/content/drive/MyDrive/Human_AI_detection/mal_test_with_labels.csv')
tam_actual_values = tam_actual_df['Label']
mal_actual_values = mal_actual_df['Label']

#indicBert
print("\nIndicBERT\n")
tam_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsindicBERT_tamil_run.tsv', sep='\t')
mal_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsindicBERT_malayalam_run.tsv', sep='\t')


tam_predicted_values = tam_predicted_df['PREDICTION']
mal_predicted_values = mal_predicted_df['PREDICTION']


# macro F1 score
tam_macro_f1 = f1_score(tam_actual_values, tam_predicted_values, average='macro')
mal_macro_f1 = f1_score(mal_actual_values, mal_predicted_values, average='macro')
# Calculate the accuracy
tam_accuracy = accuracy_score(tam_actual_values, tam_predicted_values)
mal_accuracy = accuracy_score(mal_actual_values,mal_predicted_values)

print(f"Tamil macro F1 Score : {tam_macro_f1}")
print(f"Tamil Accuracy : {tam_accuracy}")

print(f"Malayalam macro F1 Score : {mal_macro_f1}")
print(f"Malayalam Accuracy : {mal_accuracy}")


#SVM
print("\nSVM\n")
tam_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsSVM_tamil_run.tsv', sep='\t')
mal_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsSVM_malayalam_run.tsv', sep='\t')


tam_predicted_values = tam_predicted_df['PREDICTION']
mal_predicted_values = mal_predicted_df['PREDICTION']


# macro F1 score
tam_macro_f1 = f1_score(tam_actual_values, tam_predicted_values, average='macro')
mal_macro_f1 = f1_score(mal_actual_values, mal_predicted_values, average='macro')
# Calculate the accuracy
tam_accuracy = accuracy_score(tam_actual_values, tam_predicted_values)
mal_accuracy = accuracy_score(mal_actual_values,mal_predicted_values)

print(f"Tamil macro F1 Score : {tam_macro_f1}")
print(f"Tamil Accuracy : {tam_accuracy}")

print(f"Malayalam macro F1 Score : {mal_macro_f1}")
print(f"Malayalam Accuracy : {mal_accuracy}")


#AlBert
print("\nAlBERT\n")
tam_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsALBERT_tamil_run.tsv', sep='\t')
mal_predicted_df = pd.read_csv('/content/drive/MyDrive/Human_AI_detection/AnalysisArchitectsALBERT_malayalam_run.tsv', sep='\t')


tam_predicted_values = tam_predicted_df['PREDICTED_LABEL']
mal_predicted_values = mal_predicted_df['PREDICTED_LABEL']


# macro F1 score
tam_macro_f1 = f1_score(tam_actual_values, tam_predicted_values, average='macro')
mal_macro_f1 = f1_score(mal_actual_values, mal_predicted_values, average='macro')
# Calculate the accuracy
tam_accuracy = accuracy_score(tam_actual_values, tam_predicted_values)
mal_accuracy = accuracy_score(mal_actual_values,mal_predicted_values)

print(f"Tamil macro F1 Score: {tam_macro_f1}")
print(f"Tamil Accuracy : {tam_accuracy}")

print(f"Malayalam macro F1 Score : {mal_macro_f1}")
print(f"Malayalam Accuracy : {mal_accuracy}")





IndicBERT

Tamil macro F1 Score : 0.62
Tamil Accuracy : 0.62
Malayalam macro F1 Score : 0.3333333333333333
Malayalam Accuracy : 0.5

SVM

Tamil macro F1 Score : 0.6594551282051282
Tamil Accuracy : 0.66
Malayalam macro F1 Score : 0.6797117405665098
Malayalam Accuracy : 0.68

AlBERT

Tamil macro F1 Score: 0.4375
Tamil Accuracy : 0.46
Malayalam macro F1 Score : 0.8849741191768148
Malayalam Accuracy : 0.885


# **GPT MODEL**

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
import numpy as np
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

# Tokenize the text
text = "Consider this text in tamil : விலை அதிகமா இருக்கு, அளவு குறைவாக இருக்கு. Classify it based on who wrote the text: HUMAN or AI"
encoded_input = tokenizer(text, return_tensors='pt')

# Get the model's output
output = model(**encoded_input)
print(output)

label=np.argmax(output, axis=-1)

print("label: ",label)

if label==1:
  print("HUMAN")
else:
  print("AI")

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-4.9497e-02, -3.3431e-02, -3.2315e-01,  ..., -1.3112e-01,
           7.0960e-03, -1.3714e-01],
         [ 3.2554e-01,  2.2077e-02, -4.9034e-01,  ...,  1.9111e-01,
           7.7404e-02,  2.6223e-01],
         [ 4.3211e-01,  3.5441e-01, -7.3074e-01,  ...,  2.7675e-01,
           2.5425e-01,  2.3741e-01],
         ...,
         [-2.3674e-01, -1.5067e-04, -6.0535e-01,  ...,  1.0687e-01,
          -3.3675e-01, -2.0725e-01],
         [ 2.9675e-01,  2.1653e-02, -1.1905e-02,  ...,  9.4856e-02,
          -2.2957e-01,  2.1492e-01],
         [-3.7274e-01, -3.2471e-01,  3.3938e-02,  ...,  1.4709e-02,
           1.4362e-01,  3.6806e-01]]], grad_fn=<ViewBackward0>), past_key_values=((tensor([[[[-6.9740e-01,  1.2464e+00,  1.0273e+00,  ..., -1.2870e+00,
            3.5850e-01,  1.0080e+00],
          [-1.9078e+00,  2.4359e+00,  1.6746e+00,  ..., -1.2324e+00,
           -1.5769e+00,  2.0689e+00],
          [-2.4626e+00,  2.3305e+00,

In [ ]:
import torch
import numpy as np

# Tokenize the text
#text = "Consider this text in tamil : இது பேரபன் ஃப்ரீசொன்னதுனால வாங்கிட்டு இருக்கேன். Classify it based on who wrote the text: HUMAN or AI"
text = "Consider this product review in tamil : இந்த சோப்பின் மணம் மிகவும் புத்துணர்ச்சியூட்டும் வகையில் உள்ளது. Classify it based on who wrote the text: HUMAN or AI"
encoded_input = tokenizer(text, return_tensors='pt')

# Get the model's output
output = model(**encoded_input)

# Handle different output structures
if hasattr(output, 'logits'):
    logits = output.logits
else:
    logits = output[0]

# Use only the [CLS] token's logits for classification
cls_logits = logits[:, 0, :]  # Extract the first token's logits

# Convert logits to probabilities using softmax
probs = torch.nn.functional.softmax(cls_logits, dim=-1)

# Get the predicted label index
label = torch.argmax(probs, dim=-1).item()
print(label)
# Print the label
if label == 0:
    print("HUMAN")
else:
    print("AI")



AttributeError: 'BaseModelOutputWithPastAndCrossAttentions' object has no attribute 'logits'

# **GPT 2 USING A LIBRARY**

In [ ]:
pip install Text-Classifier-based-Openai-Community-gpt2

INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 

In [ ]:
!pip install Text-Classifier-based-Openai-Community-gpt2==0.1.3

In [ ]:
from Text_Classifier_based_Openai_Community_gpt2 import TextClassificationModel
# Initialize the multilingual GPT-2 Fine-tuning model
model = TextClassificationModel()

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'Cache' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [ ]:
predictions_df = model.predict(text_test, y_test,"/content/drive/MyDrive/Human_AI_detection" , "/content/drive/MyDrive/Human_AI_detection")

print("Predictions saved in the file:", predictions_df)